In [1]:
%pwd

'/workspaces/chicken-disease-classification/research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/workspaces/chicken-disease-classification'

In [4]:
import tensorflow as tf

2023-09-28 06:43:40.399115: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-28 06:43:40.430329: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-28 06:43:40.430811: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 06:43:41.255962: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

## Entities 

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    params_batch_size: int
    params_image_size: list
    all_params: dict

## configurtions manager in src/config

In [9]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories, save_json
import tensorflow as tf

In [10]:
class ConfigurationManager():
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        print("Configuration Manager Initiated")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.keras"),
            training_data=Path("artifacts/data_ingestion/Chicken-Disease-Dataset"),
            params_batch_size=self.params.BATCH_SIZE,
            params_image_size=self.params.IMAGE_SIZE,
            all_params=self.params
        )
        return eval_config


## Components

In [11]:
from urllib.parse import urlparse

In [15]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path_of_model : Path):
        return tf.keras.models.load_model(path_of_model)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator, verbose=1)

    def save_score(self):
        scores = {
            "loss" : self.score[0],
            "accuracy" : self.score[1]
            }
        save_json(path=Path("scores.json"), data=scores)


## Pipeline

In [16]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(config=val_config)
    evaluation.evaluation()
    evaluation.save_score()
    
except Exception as e:
    raise e


Configuration Manager Initiated
[ 2023-09-28 07:06:40,889: INFO: common:  yaml file: config/config.yaml loaded successfully]
[ 2023-09-28 07:06:40,891: INFO: common:  yaml file: params.yaml loaded successfully]
[ 2023-09-28 07:06:40,892: INFO: common:  directory: artifacts created successfully]
[ 2023-09-28 07:06:40,893: INFO: common:  directory: artifacts created successfully]
Found 240 images belonging to 2 classes.
15/15 [==============================] - 41s 3s/step - loss: 0.1969 - accuracy: 0.9583
[ 2023-09-28 07:07:23,566: INFO: common:  json file: scores.json saved successfully]
